# 🚀 Entraînement Llama 3.2 3B pour Générateur de Formulaires

Ce notebook permet d'entraîner un modèle Llama 3.2 3B avec LoRA pour générer des structures de formulaires JSON.

**Configuration recommandée:**
- Runtime: Python 3
- Hardware accelerator: GPU (T4, V100, ou A100)
- RAM: High-RAM si disponible

**Temps estimé:** 1-3 heures selon le GPU

## 📋 Étape 1: Configuration de l'environnement

Installation des bibliothèques nécessaires.

In [ ]:
# Installation des dépendances
!pip install -q -U transformers datasets accelerate peft bitsandbytes sentencepiece
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

print("✅ Installation terminée!")

In [ ]:
# Vérification du GPU
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("⚠️ Attention: GPU non disponible. L'entraînement sera très lent.")
    print("Allez dans Runtime > Change runtime type > Hardware accelerator > GPU")

## 💾 Étape 2: Connexion à Google Drive

Pour sauvegarder le modèle entraîné.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Créer le dossier de sauvegarde
!mkdir -p /content/drive/MyDrive/llama3-form-generator

print("✅ Google Drive monté!")

## 🔐 Étape 3: Authentification Hugging Face

Llama 3.2 3B nécessite une acceptation de la licence et un token Hugging Face.

**Instructions:**
1. Allez sur https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct
2. Cliquez sur "Agree and access repository"
3. Créez un token sur https://huggingface.co/settings/tokens
4. Collez-le ci-dessous

In [ ]:
from huggingface_hub import login
from getpass import getpass

# Entrez votre token Hugging Face
token = getpass("Entrez votre token Hugging Face: ")
login(token=token)

print("✅ Authentification réussie!")

## 📁 Étape 4: Préparation des données

Deux options:
- **Option A**: Uploader votre dataset existant
- **Option B**: Cloner le repository GitHub

In [ ]:
# Option A: Upload manuel du dataset
from google.colab import files

print("Uploadez votre fichier training_dataset.jsonl:")
uploaded = files.upload()

print("✅ Dataset uploadé!")

In [ ]:
# Option B: Cloner le repository (décommentez si vous utilisez cette option)
# !git clone https://github.com/VOTRE_USERNAME/FORM_EDITOR_LLM.git
# %cd FORM_EDITOR_LLM
# !python prepare_dataset.py
# !cp training_dataset.jsonl /content/

In [ ]:
# Vérifier le dataset
import json
import os

dataset_path = "/content/training_dataset.jsonl"

if os.path.exists(dataset_path):
    with open(dataset_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        print(f"✅ Dataset trouvé: {len(lines)} exemples")
        
        # Afficher un exemple
        if lines:
            example = json.loads(lines[0])
            print("\n📋 Exemple:")
            print(f"Instruction: {example['instruction'][:100]}...")
            print(f"Output: {example['output'][:100]}...")
else:
    print("❌ Dataset non trouvé. Veuillez uploader training_dataset.jsonl")

## 🏋️ Étape 5: Entraînement du modèle

Cette cellule lance l'entraînement avec Llama 3.2 3B.

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset

print("🚀 Démarrage de l'entraînement...\n")

# Configuration
model_name = "meta-llama/Llama-3.2-3B-Instruct"
output_dir = "/content/llama3-form-generator"

# Charger le tokenizer
print("📥 Chargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Charger le modèle
print("📥 Chargement du modèle (cela peut prendre quelques minutes)...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True
)

# Préparer pour LoRA
model = prepare_model_for_kbit_training(model)

# Configuration LoRA
print("🔧 Configuration LoRA...")
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Charger le dataset
print("\n📚 Chargement du dataset...")
dataset = load_dataset('json', data_files="/content/training_dataset.jsonl", split='train')
print(f"Dataset size: {len(dataset)} exemples")

# Format de prompt pour Llama 3.2 Instruct
def format_and_tokenize(example):
    text = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Tu es un assistant spécialisé dans la génération de structures de formulaires JSON.<|eot_id|><|start_header_id|>user<|end_header_id|>

{example['instruction']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{example['output']}<|eot_id|>"""

    return tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=2048
    )

print("🔄 Tokenisation du dataset...")
dataset = dataset.map(
    format_and_tokenize,
    remove_columns=dataset.column_names,
    desc="Tokenizing"
)
dataset = dataset.train_test_split(test_size=0.1)

print(f"Train: {len(dataset['train'])} | Test: {len(dataset['test'])}")

# Configuration d'entraînement
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    fp16=True,
    optim="paged_adamw_32bit",
    dataloader_num_workers=2,
    report_to="none",
    load_best_model_at_end=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

print("\n🏋️ Début de l'entraînement...")
print("=" * 60)

# Entraîner
trainer.train()

print("\n✅ Entraînement terminé!")

## 💾 Étape 6: Sauvegarde du modèle

In [ ]:
# Sauvegarder le modèle localement
print("💾 Sauvegarde du modèle...")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Modèle sauvegardé dans {output_dir}")

# Copier vers Google Drive
print("\n📤 Copie vers Google Drive...")
!cp -r {output_dir} /content/drive/MyDrive/

print("✅ Modèle sauvegardé sur Google Drive!")
print(f"📁 Chemin: /content/drive/MyDrive/llama3-form-generator")

## 🧪 Étape 7: Test du modèle

Testons le modèle avec un exemple.

In [ ]:
import json

def test_model(instruction):
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Tu es un assistant spécialisé dans la génération de structures de formulaires JSON.<|eot_id|><|start_header_id|>user<|end_header_id|>

{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1024,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extraire la réponse
    if "assistant" in result:
        response = result.split("assistant")[1].strip()
        return response
    return result

# Test
print("🧪 Test du modèle:\n")
instruction = "Crée un formulaire d'inscription avec nom, prénom, email et téléphone"
print(f"Instruction: {instruction}\n")

result = test_model(instruction)
print(f"Résultat:\n{result}")

## 📥 Étape 8: Télécharger le modèle (optionnel)

Si vous voulez télécharger le modèle localement.

In [ ]:
# Compresser le modèle
!zip -r llama3-form-generator.zip {output_dir}

# Télécharger
from google.colab import files
files.download('llama3-form-generator.zip')

print("✅ Téléchargement lancé!")

## 📊 Résumé

### Ce que vous avez fait:
1. ✅ Configuré l'environnement Colab
2. ✅ Authentifié avec Hugging Face
3. ✅ Chargé le dataset d'entraînement
4. ✅ Entraîné Llama 3.2 3B avec LoRA
5. ✅ Sauvegardé le modèle sur Google Drive
6. ✅ Testé le modèle

### Prochaines étapes:
- Utilisez le notebook `inference_colab.ipynb` pour générer des formulaires
- Le modèle est disponible sur votre Drive: `/content/drive/MyDrive/llama3-form-generator`
- Vous pouvez affiner l'entraînement en modifiant les hyperparamètres

### Ressources:
- [Documentation Llama 3.2](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)
- [Documentation LoRA](https://huggingface.co/docs/peft/main/en/conceptual_guides/lora)
- [Documentation Transformers](https://huggingface.co/docs/transformers/)